# Project 3 a 
## Chinagorom Mbaraonye

In [16]:
dir = "C:/Users/Chinagorom Mbaraonye/Downloads"

import numpy as np
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

#read in the file and make a copy of the dataset
csv_path = os.path.join(dir, "googleplaystore.csv")
apps = pd.read_csv(csv_path)
dat = apps.copy()

#separate features from labels
y = dat["Installs"]
X = dat.drop("Installs", axis=1)

classnames, indices = np.unique(y, return_inverse=True)
y = indices

#first split into train and test sets
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(X,y, test_size=0.2, random_state=34, 
                                                              stratify=y)

#now split the train_full into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_full,y_train_full, test_size=0.2, random_state=34, 
                                                              stratify=y_train_full)

#separate numeric from categorical features
X_num = X.select_dtypes(include=[np.number]) 
X_cat = X.select_dtypes(exclude=[np.number])

#build pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_attribs = list(X_num)

cat_classes = np.unique(dat["Category"])
type_classes = np.unique(dat["Type"])
cont_classes = np.unique(dat["Content Rating"])
gen_classes = np.unique(dat["Genres"])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat1", OneHotEncoder(categories=[cat_classes]), ["Category"]),
        ("cat2", OneHotEncoder(categories=[type_classes]), ["Type"]),
        ("cat3", OneHotEncoder(categories=[cont_classes]), ["Content Rating"]),
        ("cat4", OneHotEncoder(categories=[gen_classes]), ["Genres"])
    ])

X_train_prep = full_pipeline.fit_transform(X_train)
X_val_prep = full_pipeline.transform(X_val)
X_test_prep = full_pipeline.transform(X_test)
print(X_train_prep.getnnz)

<bound method _cs_matrix.getnnz of <6936x165 sparse matrix of type '<class 'numpy.float64'>'
	with 62424 stored elements in Compressed Sparse Row format>>


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

n_estimators = [int(i) for i in np.linspace(start=100, stop =600, num = 5) ]
max_leaf_nodes = [int(i) for i in np.linspace(start=15, stop=100, num = 5 )]


param_grid = {
    'n_estimators': n_estimators,
    'max_leaf_nodes': max_leaf_nodes
}
random_reg = RandomForestClassifier(random_state=34)
randomForest = GridSearchCV(random_reg, param_grid, cv=5,
                           scoring='accuracy',
                           return_train_score=True, verbose = 1, n_jobs=-1)

randomForest.fit(X_train_prep, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  2.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=34,
                                  

In [25]:
from sklearn.metrics import mean_squared_error

pred_model = randomForest.best_estimator_
print("Accuracy on the training data: ", pred_model.score(X_train_prep, y_train))
print("Accuracy on the test set: ", pred_model.score(X_test_prep, y_test))



Accuracy on the training data:  0.5482987312572087
Accuracy on the test set:  0.48385608856088563


#### Using the random forest, I got over a 30% increase in accuracy on the test dataset than using just a decision tree. This was probably becuase a random forest has multiple decision trees. Each of the decision  trees are based on a random sample of the dataset. A single decision tree is prone to overfit but a random forest reduces the chances of it overfitting by aggregating all the results of the many decision trees in the random forest which makes it more robust . With the decision tree there was a very big difference in accuracy between the test data and the training data(37% difference) which is a sign of overfitting. But with the random forest the difference in accuracy between the training data and test data reduces to just 6.5%, so the overfitting has deffinitely reduced